In [1]:
import monai
import torch

image_dir_no_patho = ["./data/ATLAS/splits_fid/atlas_no_patho_png.csv"]
image_dir_patho_sub1 = ["./data/ATLAS/splits_fid/atlas_patho_sub1_png.csv"]
image_dir_patho_sub2 = ["./data/ATLAS/splits_fid/atlas_patho_sub2_png.csv"]
image_dir_ixi = ["./data/ATLAS/splits_fid/ixi_png.csv"]

from torch.utils.data import DataLoader
import sys
import os

os.chdir('/home/malek/autoDDPM')
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))


# Get the parent directory
# Add the parent directory to the system path
sys.path.append(dir_path)
print(dir_path)
from data.loaders.ixi_loader import mask_preprocessing_loader, AtlasLoader, IXILoaderHarmonize, AtlasLoaderHarmonize

from PIL import Image
import numpy as np


device = "cuda" if torch.cuda.is_available() else "cpu"

mni_harmonizing = True

if mni_harmonizing:
    dataset_no_patho = AtlasLoaderHarmonize(
                image_dir_no_patho,
                target_size=(128, 128),
                test=False,
            )

    loader_no_patho = DataLoader(
        dataset_no_patho, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
    )

    dataset_patho_sub1 = AtlasLoaderHarmonize(
                image_dir_patho_sub1,
                target_size=(128, 128),
                test=False,
            )

    loader_patho_sub1 = DataLoader(
        dataset_patho_sub1, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
    )

    dataset_patho_sub2 = AtlasLoaderHarmonize(
                image_dir_patho_sub2,
                target_size=(128, 128),
                test=False,
            )

    loader_patho_sub2 = DataLoader(
        dataset_patho_sub2, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
    )
else:
    dataset_no_patho = AtlasLoader(
                image_dir_no_patho,
                target_size=(128, 128),
                test=False,
            )

    loader_no_patho = DataLoader(
        dataset_no_patho, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
    )

    dataset_patho_sub1 = AtlasLoader(
                image_dir_patho_sub1,
                target_size=(128, 128),
                test=False,
            )

    loader_patho_sub1 = DataLoader(
        dataset_patho_sub1, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
    )

    dataset_patho_sub2 = AtlasLoader(
                image_dir_patho_sub2,
                target_size=(128, 128),
                test=False,
            )

    loader_patho_sub2 = DataLoader(
        dataset_patho_sub2, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
    )



atlas_loading = False

if atlas_loading:
    dataset_ixi = AtlasLoader(
                image_dir_ixi,
                target_size=(128, 128),
                test=False,
            )
else:
    dataset_ixi = IXILoaderHarmonize(
                image_dir_ixi,
                target_size=(128, 128),
                test=False,
            )


loader_ixi = DataLoader(
    dataset_ixi, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
)


/home/malek/miniconda3/envs/cfgen/lib/python3.8/site-packages/monai/utils/tf32.py:66: UserWarning: torch.backends.cuda.matmul.allow_tf32 = True by default.
  This value defaults to True when PyTorch version in [1.7, 1.11] and may affect precision.
  See https://docs.monai.io/en/latest/precision_accelerating.html#precision-and-accelerating
  warnings.warn(


/home/malek


In [2]:
from generative.metrics import FIDMetric, MMDMetric, MultiScaleSSIMMetric, SSIMMetric

radnet = torch.hub.load("Warvito/radimagenet-models:main", model="radimagenet_resnet50",verbose=True)

radnet.to(device)
radnet.eval()

Using cache found in /home/malek/.cache/torch/hub/Warvito_radimagenet-models_main


ResNet50(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1.001e-05, momentum=0.01, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1.001e-05, momentum=0.99, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1.001e-05, momentum=0.99, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn3): BatchNorm2d(256, eps=1.001e-05, momentum=0.99, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm

In [3]:
def subtract_mean(x: torch.Tensor) -> torch.Tensor:
    mean = [0.406, 0.456, 0.485]
    x[:, 0, :, :] -= mean[0]
    x[:, 1, :, :] -= mean[1]
    x[:, 2, :, :] -= mean[2]
    return x


def spatial_average(x: torch.Tensor, keepdim: bool = True) -> torch.Tensor:
    return x.mean([2, 3], keepdim=keepdim)


def get_features(image):
    # If input has just 1 channel, repeat channel to have 3 channels
    if image.shape[1]:
        image = image.repeat(1, 3, 1, 1)

    # Change order from 'RGB' to 'BGR'
    image = image[:, [2, 1, 0], ...]

    # Subtract mean used during training
    image = subtract_mean(image)

    # Get model outputs
    with torch.no_grad():
        feature_image = radnet.forward(image)
        # flattens the image spatially
        feature_image = spatial_average(feature_image, keepdim=False)

    return feature_image


In [4]:
for (data_1,data_2,data_3,data_4) in zip(loader_no_patho,loader_patho_sub1,loader_patho_sub2,loader_ixi):
    images_no_patho = data_1[0].to(device)
    images_patho_sub1 = data_2[0].to(device)
    images_patho_sub2 = data_3[0].to(device)
    images_ixi = data_4[0].to(device)

    print(images_no_patho.shape)
    print(images_patho_sub1.shape)
    print(images_patho_sub2.shape)
    print(images_ixi.shape)



shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128

In [5]:
synth_features = []
real_features = []

real_eval_feats = get_features(images_no_patho)
real_features.append(real_eval_feats)

# Get the features for the synthetic data
synth_eval_feats = get_features(images_patho_sub1)
synth_features.append(synth_eval_feats)

synth_features = torch.vstack(synth_features)
real_features = torch.vstack(real_features)

fid = FIDMetric()
fid_res = fid(synth_features, real_features)

print('FID score between pathological and non pathological Atlas subsets')
print(f"FID Score: {fid_res.item():.2f}")

FID score between pathological and non pathological Atlas subsets
FID Score: 3.63


In [6]:
synth_features = []
real_features = []

real_eval_feats = get_features(images_patho_sub2)
real_features.append(real_eval_feats)

# Get the features for the synthetic data
synth_eval_feats = get_features(images_patho_sub1)
synth_features.append(synth_eval_feats)
synth_features = torch.vstack(synth_features)
real_features = torch.vstack(real_features)

fid = FIDMetric()
fid_res = fid(synth_features, real_features)

print('FID score between pathological Atlas subsets')
print(f"FID Score: {fid_res.item():.2f}")

FID score between pathological Atlas subsets
FID Score: 2.65


In [7]:
synth_features = []
real_features = []

real_eval_feats = get_features(images_no_patho)
real_features.append(real_eval_feats)

# Get the features for the synthetic data
synth_eval_feats = get_features(images_ixi)
synth_features.append(synth_eval_feats)

synth_features = torch.vstack(synth_features)
real_features = torch.vstack(real_features)

fid = FIDMetric()
fid_res = fid(synth_features, real_features)
print('FID score between non pathological Atlas subset and IXI dataset')
print(f"FID Score: {fid_res.item():.2f}")

FID score between non pathological Atlas subset and IXI dataset
FID Score: 9.85


In [8]:
synth_features = []
real_features = []

real_eval_feats = get_features(images_patho_sub1)
real_features.append(real_eval_feats)

# Get the features for the synthetic data
synth_eval_feats = get_features(images_ixi)
synth_features.append(synth_eval_feats)

synth_features = torch.vstack(synth_features)
real_features = torch.vstack(real_features)

fid = FIDMetric()
fid_res = fid(synth_features, real_features)

print('FID score between pathological Atlas subset and IXI dataset')
print(f"FID Score: {fid_res.item():.2f}")

FID score between pathological Atlas subset and IXI dataset
FID Score: 12.80
